In [1]:
import ROOT
import sys

Welcome to JupyROOT 6.26/00


## Let's load the previously generated file

In [2]:
fin = ROOT.TFile("WorkspaceFile.root")
ws = fin.Get("ws")
ws.Print()


RooWorkspace(ws) ws contents

variables
---------
(background,lam,mass,signal)

p.d.f.s
-------
RooExponential::expo[ x=mass c=lam ] = 0.986073
RooHistPdf::sighisto[ pdfObs=(mass) ] = 0
RooAddPdf::sumPDF[ signal * sighisto + background * expo ] = 0.786755

datasets
--------
RooDataSet::extPDFData(mass)

embedded datasets (in pdfs and functions)
-----------------------------------------
RooDataHist::dh(mass)



In [3]:
SBModel=ROOT.RooStats.ModelConfig()
SBModel.SetWorkspace(ws)
SBModel.SetPdf("sumPDF")
SBModel.SetName("SBModel")


poi = ROOT.RooArgSet(ws.var("signal"))#setting the parameter of interest

SBModel.SetParametersOfInterest(poi)
    
#for CLs (bounded intervals) use one-sided profile likelihood
profll = ROOT.RooStats.ProfileLikelihoodTestStat(SBModel.GetPdf())
profll.SetOneSided(1)
observables=ROOT.RooArgSet((ws.var("mass")))#we need to define the observable for the fit
SBModel.SetObservables(observables)
SBModel.Print()
    
BModel=SBModel.Clone("BOnlyModel")
BModel.SetParametersOfInterest(poi)
poi.find("signal").setVal(0.01)
BModel.SetSnapshot(poi) 
BModel.SetObservables(observables)
BModel.Print()


=== Using the following for SBModel ===
Observables:             RooArgSet:: = (mass)
Parameters of Interest:  RooArgSet:: = (signal)
PDF:                     RooAddPdf::sumPDF[ signal * sighisto + background * expo ] = 0.786755


=== Using the following for BOnlyModel ===
Observables:             RooArgSet:: = (mass)
Parameters of Interest:  RooArgSet:: = (signal)
PDF:                     RooAddPdf::sumPDF[ signal * sighisto + background * expo ] = 0.986063
Snapshot:                
  1) 0x556144de9640 RooRealVar:: signal = 0.01 +/- 24.3435  L(0 - 2000)  "signal events"



In [4]:
#define the dataset to test:
AllData=(ws.data("extPDFData")) #Full dataset
AllData.Print()
ROOT.RooStats.AsymptoticCalculator.SetPrintLevel(2)


RooDataSet::extPDFData[mass] = 1250 entries


In [5]:
def printLimits(DATA,SM,BM,poiname,paramsscan=None,valuepoi=None,signif=False,asymp=False,freq=False,postfix=""):
    BM.Print()
    SM.Print()
    if ( paramsscan is None):
        paramsscan ={"npoints":8,"poimin":0,"poimax":2000,"minplot":-1,"maxplot":-1}

    npoints=paramsscan["npoints"]
    poimin=paramsscan["poimin"]
    poimax=paramsscan["poimax"]
    minplot=paramsscan["minplot"]
    maxplot=paramsscan["maxplot"]
 
    ac= ROOT.RooStats.AsymptoticCalculator(DATA, BM, SM);
    ac.Initialize()

    fc= ROOT.RooStats.FrequentistCalculator(DATA, BM, SM);
    fc.SetToys(2500,500)


    if(signif):
        ahypo= ac.GetHypoTest()
        ahypo.Print()

    if(freq):
            
        #use frequentist approach: 
        #Warning: this will take a lot of time! 
        #In general: better use asymptotic approximation if number of events is rather large. 
        #Bias in the test statistic negative log likelihood, i.e. q0, will go as 1/sqrt(N) --> negligible for large datasets

        #Create hypotest inverter passing the calculator 
        freqCalc = ROOT.RooStats.HypoTestInverter(fc)

        #set confidence level (e.g. 90% upper limits)
        freqCalc.SetConfidenceLevel(0.90)

        #use CLs
        freqCalc.UseCLs(1)
 
        #Configure ToyMC Sampler
        toymcs = freqCalc.GetHypoTestCalculator().GetTestStatSampler()

        #choose the test statistics: profile NLL
        profll = ROOT.RooStats.ProfileLikelihoodTestStat(SM.GetPdf())

        #for CLs (bounded intervals) use one-sided profile likelihood
        profll.SetOneSided(1)

        #set the test statistic to use for toys
        toymcs.SetTestStatistic(profll)

          
        #generate pseudoexperiments to get the distribution of the test statistic
            
        freqCalc.SetFixedScan(npoints,poimin,poimax);
            
        result = freqCalc.GetInterval() #This is a HypoTestInveter class object
        result.Print()
        upperLimit = result.UpperLimit()
        exp=result.GetExpectedUpperLimit(0)
        p1=result.GetExpectedUpperLimit(1)
        m1=result.GetExpectedUpperLimit(-1)
        
        print ("====\n upper limit is ", upperLimit," exp ", exp, " p1 ",p1, " m1 ",m1, "\n\n ====")
        result.Print()
        c1=ROOT.TCanvas("limitCanvas") 
        c1.SetLogy()
        inv_plot = ROOT.RooStats.HypoTestInverterPlot("HTI_Result_Plot","result plot",result)
        if minplot!=-1 and maxplot!=-1:
            gr1 = inv_plot.MakePlot();
            gr1.Draw("ALP")
            gr1.GetYaxis().SetRangeUser(minplot,maxplot)
            gr2 = inv_plot.MakeExpectedPlot();
            gr2.Draw("LF")
            gr1.Draw("LP")

        inv_plot.Draw("2CL")
        c1.SaveAs("limitex"+postfix+".png")
            
    if(asymp):
        asympCalc = ROOT.RooStats.HypoTestInverter(ac)
        asympCalc.SetConfidenceLevel(0.90)
        #use CLs
        asympCalc.UseCLs(1)
        asympCalc.SetVerbose(3)

            
        asympCalc.SetFixedScan(npoints,poimin,poimax);
            
 
        result = asympCalc.GetInterval() #This is a HypoTestInveter class object
        result = asympCalc.GetInterval() #This is a HypoTestInveter class object
        result.Print()
        upperLimit = result.UpperLimit()
        exp=result.GetExpectedUpperLimit(0)
        p1=result.GetExpectedUpperLimit(1)
        m1=result.GetExpectedUpperLimit(-1)
        
        print ("====\n upper limit is ", upperLimit," exp ", exp, " p1 ",p1, " m1 ",m1, "\n\n ====")
        result.Print()
        c1=ROOT.TCanvas("limitCanvas") 
        c1.SetLogy()
        inv_plot = ROOT.RooStats.HypoTestInverterPlot("HTI_Result_Plot","result plot",result)
        if minplot!=-1 and maxplot!=-1:
            gr1 = inv_plot.MakePlot();
            gr1.Draw("ALP")
            gr1.GetYaxis().SetRangeUser(minplot,maxplot)
            gr2 = inv_plot.MakeExpectedPlot();
            gr2.Draw("LF")
            gr1.Draw("LP")

            inv_plot.Draw("2CL")
        else:
            inv_plot.Draw("2CL")
        c1.SaveAs("limitex"+postfix+".png")


In [6]:
doFreq=False
paramsscan_alt ={"npoints":20,"poimin":0,"poimax":500,"minplot":0.001,"maxplot":10}
paramsscan_all ={"npoints":20,"poimin":100,"poimax":600,"minplot":0.001,"maxplot":10}
if doFreq:
    paramsscan_alt ={"npoints":8,"poimin":0,"poimax":500,"minplot":0.001,"maxplot":10}
    paramsscan_all ={"npoints":8,"poimin":100,"poimax":600,"minplot":0.001,"maxplot":10}
    

In [7]:
doHypothesisTest=True
doAsymp=True
if(doHypothesisTest):
    poi.find("signal").setVal(250)
    SBModel.SetSnapshot(poi)
    
printLimits(DATA=AllData,BM=BModel,SM=SBModel,poiname="signal",signif=doHypothesisTest,asymp=doAsymp,paramsscan=paramsscan_all,postfix="all")

====
 upper limit is  298.96993565016766  exp  0.0  p1  0.0  m1  0.0 

 ====

=== Using the following for BOnlyModel ===
Observables:             RooArgSet:: = (mass)
Parameters of Interest:  RooArgSet:: = (signal)
PDF:                     RooAddPdf::sumPDF[ signal * sighisto + background * expo ] = 0.788437
Snapshot:                
  1) 0x5561451518e0 RooRealVar:: signal = 0.01 +/- 24.3435  L(0 - 2000)  "signal events"


=== Using the following for SBModel ===
Observables:             RooArgSet:: = (mass)
Parameters of Interest:  RooArgSet:: = (signal)
PDF:                     RooAddPdf::sumPDF[ signal * sighisto + background * expo ] = 0.788437
Snapshot:                
  1) 0x55613ca9a740 RooRealVar:: signal = 250 +/- 24.3435  L(0 - 2000)  "signal events"

[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize....
[#0] PROGRESS:Eval -- AsymptoticCalculator::Initialize - Find  best unconditional NLL on observed data
[#1] INFO:Minimization -- p.d.f. provides expected number of events

Info in <ROOT::Math::BrentMethods::MinimStep>: Grid search failed to find a root in the  interval 
Info in <ROOT::Math::BrentMethods::MinimStep>: xmin = 0 xmax = 20 npts = 100
Error in <ROOT::Math::BrentRootFinder>: Interval does not contain a root
Info in <ROOT::Math::BrentMethods::MinimStep>: Grid search failed to find a root in the  interval 
Info in <ROOT::Math::BrentMethods::MinimStep>: xmin = 0 xmax = 20 npts = 100
Error in <ROOT::Math::BrentRootFinder>: Interval does not contain a root
Info in <ROOT::Math::BrentMethods::MinimStep>: Grid search failed to find a root in the  interval 
Info in <ROOT::Math::BrentMethods::MinimStep>: xmin = 0 xmax = 20 npts = 100
Error in <ROOT::Math::BrentRootFinder>: Interval does not contain a root
Info in <ROOT::Math::BrentMethods::MinimStep>: Grid search failed to find a root in the  interval 
Info in <ROOT::Math::BrentMethods::MinimStep>: xmin = 0 xmax = 20 npts = 100
Error in <ROOT::Math::BrentRootFinder>: Interval does not contain a root
Info

In [8]:
SmallSigData=ws.pdf("sighisto").generate(ws.var("mass"),20)
SmallSigAllData=ws.pdf("expo").generate(ws.var("mass"),1000)
SmallSigAllData.append(SmallSigData)
SBModelSmallSig=ROOT.RooStats.ModelConfig()
SBModelSmallSig.SetWorkspace(ws)
SBModelSmallSig.SetParametersOfInterest(poi)
SBModelSmallSig.SetPdf(ws.pdf("sumPDF"))
BModelSmallSig=SBModelSmallSig.Clone("BModelSmallSig")
BModelSmallSig.SetParametersOfInterest(poi)
poi.find("signal").setVal(0.01)
BModelSmallSig.SetSnapshot(poi) 
BModelSmallSig.SetObservables(observables)
BModelSmallSig.Print()


=== Using the following for BModelSmallSig ===
Observables:             RooArgSet:: = (mass)
Parameters of Interest:  RooArgSet:: = (signal)
PDF:                     RooAddPdf::sumPDF[ signal * sighisto + background * expo ] = 0.0092217/1
Snapshot:                
  1) 0x556146224aa0 RooRealVar:: signal = 0.01 +/- 24.3376  L(0 - 2000)  "signal events"

